In [1]:
'''
A bunch of imports, you don't have to worry about these
'''

import numpy as np
import random
import torch
import torch.nn as nn  
import torch.nn.functional as F
from collections import namedtuple, deque
import torch.optim as optim
import datetime
import gym
import matplotlib.pyplot as plt

In [2]:
'''
Please refer to the first tutorial for more details on the specifics of environments
We've only added important commands you might find useful for experiments.
'''

'''
List of example environments
(Source - https://gym.openai.com/envs/#classic_control)

'Acrobot-v1'
'CartPole-v0'
'MountainCar-v0'
'''

env = gym.make('Acrobot-v1')
env.seed(0)

state_shape = env.observation_space.shape[0]
no_of_actions = env.action_space.n

print(state_shape)
print(no_of_actions)
print(env.action_space.sample())
print("----")

'''
# Understanding State, Action, Reward Dynamics

The agent decides an action to take depending on the state.

The Environment keeps a variable specifically for the current state.
- Everytime an action is passed to the environment, it calculates the new state and updates the current state variable.
- It returns the new current state and reward for the agent to take the next action

'''

state = env.reset()   
''' This returns the initial state (when environment is reset) '''

print(state)
print("----")

action = env.action_space.sample()  
''' We take a random action now '''

print(action)
print("----")

next_state, reward, done, info = env.step(action) 
''' env.step is used to calculate new state and obtain reward based on old state and action taken  ''' 

print(next_state)
print(reward)
print(done)
print(info)
print("----")


6
3
1
----
[ 0.99962485  0.02738891  0.9989402  -0.04602639 -0.09180529 -0.09669447]
----
0
----
[ 0.9998245   0.01873245  0.995746   -0.09214022  0.00529764 -0.3585254 ]
-1.0
False
{}
----


In [3]:
'''
### Q Network & Some 'hyperparameters'

QNetwork1:
Input Layer - 4 nodes (State Shape) \
Hidden Layer 1 - 64 nodes \
Hidden Layer 2 - 64 nodes \
Output Layer - 2 nodes (Action Space) \
Optimizer - zero_grad()

QNetwork2: Feel free to experiment more
'''

import torch
import torch.nn as nn  
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


'''
Bunch of Hyper parameters (Which you might have to tune later **wink wink**)
'''
BUFFER_SIZE = int(5e4)  
'''
replay buffer size 
'''
BATCH_SIZE = 256         
''' 
minibatch size 
'''
GAMMA = 0.99            
''' 
discount factor 
'''
LR = 6e-4              
''' 
learning rate 
'''
UPDATE_EVERY = 250       
''' 
how often to update the network (When Q target is present) 
'''


class QNetwork1(nn.Module):

    def __init__(self, state_size, action_size, seed, fc1_units=128, fc2_units=64,fc3_units=32):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(QNetwork1, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.tanh(self.fc1(state))
        x = F.tanh(self.fc2(x))
        return self.fc3(x)

### Replay Buffer:

This is a 'deque' that helps us store experiences. Recall why we use such a technique.

In [4]:
import random
import torch
import numpy as np
from collections import deque, namedtuple

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(device))

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

NVIDIA GeForce GTX 1660 Ti


## Truncation:
We add a line (optionally) in the code to truncate the gradient in hopes that it would help with the stability of the learning process.

## Tutorial Agent Code:


In [5]:
class TutorialAgent():

    def __init__(self, state_size, action_size, seed):

        ''' Agent Environment Interaction '''
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        ''' Q-Network '''
        self.qnetwork_local = QNetwork1(state_size, action_size, seed,128,128).to(device)
        self.qnetwork_target = QNetwork1(state_size, action_size, seed,128,128).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        ''' Replay memory '''
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

        ''' Initialize time step (for updating every UPDATE_EVERY steps)           -Needed for Q Targets '''
        self.t_step = 0
        self.rg = np.random.RandomState(seed)
    
    def step(self, state, action, reward, next_state, done):

        ''' Save experience in replay memory '''
        self.memory.add(state, action, reward, next_state, done)
        
        ''' If enough samples are available in memory, get random subset and learn '''
        if len(self.memory) >= BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

        """ +Q TARGETS PRESENT """
        ''' Updating the Network every 'UPDATE_EVERY' steps taken '''      
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:

            self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())

    def act(self, state, eps=0.):

        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        ''' Epsilon-greedy action selection (Already Present) '''
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
#         probabs = softmax(action_values.cpu().data.numpy()[0])
#         probabs /= probabs.sum()
#         return self.rg.choice(np.arange(self.action_size), p = probabs)

    def learn(self, experiences, gamma):
        """ +E EXPERIENCE REPLAY PRESENT """
        states, actions, rewards, next_states, dones = experiences

        ''' Get max predicted Q values (for next states) from target model'''
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)

        ''' Compute Q targets for current states '''
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        ''' Get expected Q values from local model '''
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        ''' Compute loss '''
        loss = F.mse_loss(Q_expected, Q_targets)

        ''' Minimize the loss '''
        self.optimizer.zero_grad()
        loss.backward()
        
        ''' Gradiant Clipping '''
        """ +T TRUNCATION PRESENT """
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
            
        self.optimizer.step()

### Here, we present the DQN algorithm code.

In [6]:
solved_episodes = []
rewards = []

In [7]:
''' Defining DQN Algorithm '''
for i in range(10) :
    state_shape = env.observation_space.shape[0]
    action_shape = env.action_space.n
    reward_list = []
    def dqn(n_episodes=10000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.99):

        scores = []                 
        ''' list containing scores from each episode '''

        scores_window_printing = deque(maxlen=10) 
        ''' For printing in the graph '''

        scores_window= deque(maxlen=100)  
        ''' last 100 scores for checking if the avg is more than 195 '''

        eps = eps_start                    
        ''' initialize epsilon '''

        for i_episode in range(1, n_episodes+1):
            state = env.reset()
            score = 0
            for t in range(max_t):
                action = agent.act(state, eps)
                next_state, reward, done, _ = env.step(action)
                agent.step(state, action, reward, next_state, done)
                state = next_state
                score += reward
                if done:
                    break 

            reward_list.append(score)
            scores_window.append(score)       
            scores_window_printing.append(score)   
            ''' save most recent score '''           

            eps = max(eps_end, eps_decay*eps) 
            ''' decrease epsilon '''

            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")  
            if i_episode % 10 == 0: 
                scores.append(np.mean(scores_window_printing))        
            if i_episode % 100 == 0: 
               print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            if np.mean(scores_window)>=-80.0:
               solved_episodes.append(i_episode)
               print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
               break
        return [np.array(scores),i_episode-100]

    ''' Trial run to check if algorithm runs and saves the data '''

    begin_time = datetime.datetime.now()
    agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0)


    dqn()


    time_taken = datetime.datetime.now() - begin_time

    print(time_taken)
    rewards.append(reward_list)

C:\Users\Vedant\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 100	Average Score: -262.53
Episode 200	Average Score: -114.80
Episode 300	Average Score: -90.489
Episode 400	Average Score: -81.30
Episode 441	Average Score: -79.91
Environment solved in 341 episodes!	Average Score: -79.91
0:10:18.035278
Episode 100	Average Score: -261.67
Episode 200	Average Score: -109.82
Episode 300	Average Score: -91.277
Episode 400	Average Score: -83.33
Episode 456	Average Score: -79.53
Environment solved in 356 episodes!	Average Score: -79.53
0:10:43.131236
Episode 100	Average Score: -285.21
Episode 200	Average Score: -109.61
Episode 300	Average Score: -88.290
Episode 400	Average Score: -81.35
Episode 411	Average Score: -79.86
Environment solved in 311 episodes!	Average Score: -79.86
0:10:27.132510
Episode 100	Average Score: -249.90
Episode 200	Average Score: -107.73
Episode 300	Average Score: -86.663
Episode 400	Average Score: -81.48
Episode 434	Average Score: -79.84
Environment solved in 334 episodes!	Average Score: -79.84
0:09:58.547811
Episode 100	Aver

In [ ]:
rewards1=rewards
average_reward_list = []
for i in range(10000) :
        average_reward_list.append(0)
for j in range(10) :
    for i in range(len(rewards[j])) :
        average_reward_list[i]+=rewards[j][i]
    for k in range(i,10000) :
        average_reward_list[k]+=rewards[j][i]
for i in range(len(average_reward_list)) :
    average_reward_list[i]/=10

In [ ]:
plt.plot(average_reward_list)

In [ ]:
average_reward_list1 = []
for i in range(100,len(average_reward_list)) :
  average_reward_list1.append(np.average(average_reward_list[i-100:i]))
plt.plot(average_reward_list1)

In [ ]:
print(np.mean(solved_episodes))
print(solved_episodes)
print(np.mean(solved_episodes))
print(rewards)